In [2]:
import tensorflow as tf
import os
from matplotlib import pyplot as plt
import numpy as np
%matplotlib inline

In [9]:
TRAIN_DIR = "train/"
TEST_DIR = "test/"

In [10]:
train_image_file_names = [TRAIN_DIR+i for i in os.listdir(TRAIN_DIR)][0:3000] 
test_image_file_names = [TEST_DIR+i for i in os.listdir(TEST_DIR)][0:1000]

In [11]:
def resize_image(image_file_names, height, width):
    images = []
    
    graph = tf.Graph()
    with graph.as_default():
        file_name = tf.placeholder(dtype=tf.string)
        file = tf.read_file(file_name)
        image = tf.image.decode_jpeg(file)
        image = tf.image.resize_image_with_crop_or_pad(image, height, width)
    
    with tf.Session(graph=graph) as session:
        tf.global_variables_initializer()  
        for i in range(len(image_file_names)):
            images.append(session.run(image, feed_dict={file_name: image_file_names[i]}))
            if (i+1) % 1000 == 0:
                print('Images processed: ',i+1)
        
        session.close()
    
    return images

In [12]:
resize_image(train_image_file_names,500,500)

Images processed:  1000
Images processed:  2000
Images processed:  3000


[array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        ..., 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        [[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]]], dtype=uint8), array([[[0, 0, 0],
         [0, 0, 0],
         [0, 0, 0],
         ..., 
         [0, 0, 0],
         [0, 0, 0],
         [0, 0, 0]],
 
        